# Search through gnps libraries to select dataset

Starting from gnps public libraries (speclibs folders, version from 5 July 2019), we here create one large MGF file with all spectra.
In addition Smiles or Inchi's are added from metadata file.

In [1]:
# Import
import json
import os

In [2]:
# Locations
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_Wageningen_iOMEGA"
PATH_MS_DATA = ROOT + "\\Data\\GNPS_all\\"
PATH_SAVE_MODEL = ROOT + "\\Spec2Vec\\models_trained\\"
PATH_SAVE_DATA = ROOT + "\\Spec2Vec\\data\\"
PATH_SPEC2VEC = ROOT + "\\Spec2Vec\\"

## Importing additional metadata

In [3]:
json_file = PATH_MS_DATA + 'gnpslibraryjson_filtered.json'

with open(json_file, 'rb') as f:
    data = json.load(f)

In [4]:
print("Metadata available for", len(data), "spectra.")

Metadata available for 77253 spectra.


In [5]:
gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry

## Briefly inspect metadata

In [6]:
keys = []
for key in gnps_data.keys():
    keys.append(key)

In [7]:
keys[0]

'CCMSLIB00000001548'

In [8]:
gnps_data[keys[0]]

{'COMPOUND_INCHIKEY': 'KNGPFNUOXXLKCN-ZNCJFREWSA-N',
 'COMPOUND_INCHI': 'InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(8)37-25(7)30(58-13)18-31-46-29(19-61-31)39-49-45(12,21-62-39)43-50-44(11,20-63-43)42(57)48-32(22(4)15-2)35(53)27(9)40(55)59-36(23(5)16-3)38(54)47-33(28(10)51)41(56)60-37/h19,22-28,30,32-37,51-53H,14-18,20-21H2,1-13H3,(H,47,54)(H,48,57)/t22-,23-,24+,25-,26-,27+,28+,30-,32-,33-,34-,35-,36-,37-,44+,45+/m0/s1',
 'LIBRARY_QUALITY': '1',
 'GNPSID': 'CCMSLIB00000001548',
 'COMPOUND_NAME': 'Hoiamide B',
 'COMPOUND_SMILES': 'CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](Cc2nc(cs2)C3=N[C@](CS3)(C4=N[C@](CS4)(C(=O)N[C@H]([C@H]([C@H](C(=O)O[C@H](C(=O)N[C@H](C(=O)O1)[C@@H](C)O)[C@@H](C)CC)C)O)[C@@H](C)CC)C)C)OC)C)O'}

## Functions

In [9]:
def check_spectrum_lines(spectrum):
    """ Check if desired metadata is present for spectrum.
    """
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    """ Add metadata to spectrum (taken from json file, see above).
    """
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename, matching=True):
    """ Collect spectra in file (line by line). Check if matching metadata is available.
    
    matching: bool
        If matching = True than only include spectra for which desired metadata is present.
        For matching = False, include all spectra.
    """
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            
            if line.startswith('END ION'):
                
                if check_spectrum_lines(spectrum): # Check if we have structural info
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
                else: 
                    if not matching: # then include all spectra
                        matched_spectra.append(spectrum)
    return matched_spectra

## Create MGF for all spectra (also non matched ones)

In [10]:
path = PATH_MS_DATA + 'speclibs\\'
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f), matching=False)
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))

BILELIB19.mgf 177
CASMI.mgf 745
DEREPLICATOR_IDENTIFIED_LIBRARY.mgf 1124
GNPS-COLLECTIONS-MISC.mgf 1170
GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf 1246
GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf 1899
GNPS-EMBL-MCF.mgf 2484
GNPS-FAULKNERLEGACY.mgf 2611
GNPS-LIBRARY.mgf 7394
GNPS-NIH-CLINICALCOLLECTION1.mgf 7771
GNPS-NIH-CLINICALCOLLECTION2.mgf 7966
GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf 9233
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf 11096
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf 16892
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf 18352
GNPS-NIST14-MATCHES.mgf 24115
GNPS-PRESTWICKPHYTOCHEM.mgf 24258
GNPS-SELLECKCHEM-FDA-PART1.mgf 26646
GNPS-SELLECKCHEM-FDA-PART2.mgf 27302
HMDB.mgf 29537
LDB_NEGATIVE.mgf 29763
LDB_POSITIVE.mgf 29846
MASSBANK.mgf 41845
MASSBANKEU.mgf 43337
MIADB.mgf 43509
MMV_NEGATIVE.mgf 43556
MMV_POSITIVE.mgf 43666
MONA.mgf 92907
PNNL-LIPIDS-NEGATIVE.mgf 109049
PNNL-LIPIDS-POSITIVE.mgf 139631
RESPECT.mgf 146743
SUMNER.mgf 147647


In [11]:
len(collected_spectra)

147647

# Convert smiles into inchi where necessary

In [43]:
from rdkit import Chem

def smiles_to_inchi(smiles):
    mol = Chem.MolFromSmiles(smiles)
    try: 
        inchi = Chem.inchi.MolToInchi(mol)
    except:
        print("Error for smiles:", smiles)
        inchi = "N/A"
    return inchi

In [60]:
for i, spectrum in enumerate(collected_spectra):
    for l in spectrum:
        if l.startswith('SMILES'):
            smiles = l.strip().split('SMILES=')[1]
        if l.startswith('INCHI='):
            inchi = l.strip().split('=')[1]
            
    # Look for cases where inchi is missing but smiles is present       
    if inchi is None \
        or inchi in ['N/A', 'NA', 0, '0'] \
        or len(inchi) == 0:
            if smiles is not None \
                and smiles not in ['N/A', 'NA', 0, '0', 'InChI'] \
                and len(smiles) > 0:
                    spectrum_new = []
                    for l in spectrum:
                        if l.startswith('INCHI='):
                            found_inchi = smiles_to_inchi(smiles)
                            l = 'INCHI=%s\n' % found_inchi
                        spectrum_new.append(l)
                    collected_spectra[i] = spectrum_new
                    print("Added inchi to spectrum", i)

Added inchi to spectrum 745
Added inchi to spectrum 746
Added inchi to spectrum 747
Added inchi to spectrum 748
Added inchi to spectrum 749
Added inchi to spectrum 750
Added inchi to spectrum 751
Added inchi to spectrum 752
Added inchi to spectrum 753
Added inchi to spectrum 754
Added inchi to spectrum 755
Added inchi to spectrum 756
Added inchi to spectrum 757
Added inchi to spectrum 758
Added inchi to spectrum 759
Added inchi to spectrum 760
Added inchi to spectrum 761
Added inchi to spectrum 762
Added inchi to spectrum 763
Added inchi to spectrum 764
Added inchi to spectrum 765
Added inchi to spectrum 766
Added inchi to spectrum 767
Added inchi to spectrum 768
Added inchi to spectrum 769
Added inchi to spectrum 770
Added inchi to spectrum 771
Added inchi to spectrum 772
Added inchi to spectrum 773
Added inchi to spectrum 774
Added inchi to spectrum 775
Added inchi to spectrum 776
Added inchi to spectrum 777
Added inchi to spectrum 778
Added inchi to spectrum 779
Added inchi to spect

Added inchi to spectrum 1085
Added inchi to spectrum 1086
Added inchi to spectrum 1087
Added inchi to spectrum 1088
Added inchi to spectrum 1089
Added inchi to spectrum 1090
Added inchi to spectrum 1091
Added inchi to spectrum 1092
Added inchi to spectrum 1093
Added inchi to spectrum 1094
Added inchi to spectrum 1095
Added inchi to spectrum 1096
Added inchi to spectrum 1097
Added inchi to spectrum 1098
Added inchi to spectrum 1099
Added inchi to spectrum 1100
Added inchi to spectrum 1101
Added inchi to spectrum 1102
Added inchi to spectrum 1103
Added inchi to spectrum 1104
Added inchi to spectrum 1105
Added inchi to spectrum 1106
Added inchi to spectrum 1107
Added inchi to spectrum 1108
Added inchi to spectrum 1109
Added inchi to spectrum 1110
Added inchi to spectrum 1111
Added inchi to spectrum 1112
Added inchi to spectrum 1113
Added inchi to spectrum 1114
Added inchi to spectrum 1115
Added inchi to spectrum 1116
Added inchi to spectrum 1117
Added inchi to spectrum 1118
Added inchi to

Added inchi to spectrum 2413
Added inchi to spectrum 2414
Added inchi to spectrum 2415
Added inchi to spectrum 2416
Added inchi to spectrum 2417
Added inchi to spectrum 2418
Added inchi to spectrum 2419
Added inchi to spectrum 2420
Added inchi to spectrum 2421
Added inchi to spectrum 2422
Added inchi to spectrum 2423
Added inchi to spectrum 2424
Added inchi to spectrum 2425
Added inchi to spectrum 2426
Added inchi to spectrum 2427
Added inchi to spectrum 2428
Added inchi to spectrum 2429
Added inchi to spectrum 2430
Added inchi to spectrum 2431
Added inchi to spectrum 2432
Added inchi to spectrum 2433
Added inchi to spectrum 2434
Added inchi to spectrum 2435
Added inchi to spectrum 2436
Added inchi to spectrum 2437
Added inchi to spectrum 2438
Added inchi to spectrum 2439
Added inchi to spectrum 2440
Added inchi to spectrum 2441
Added inchi to spectrum 2442
Added inchi to spectrum 2443
Added inchi to spectrum 2444
Added inchi to spectrum 2445
Added inchi to spectrum 2446
Added inchi to

Added inchi to spectrum 4970
Added inchi to spectrum 4971
Added inchi to spectrum 4972
Added inchi to spectrum 4973
Added inchi to spectrum 4974
Added inchi to spectrum 4975
Added inchi to spectrum 4976
Added inchi to spectrum 4977
Added inchi to spectrum 4978
Added inchi to spectrum 4979
Added inchi to spectrum 4980
Added inchi to spectrum 5001
Added inchi to spectrum 5002
Added inchi to spectrum 5003
Added inchi to spectrum 5004
Added inchi to spectrum 5005
Added inchi to spectrum 5006
Added inchi to spectrum 5007
Added inchi to spectrum 5052
Added inchi to spectrum 5100
Added inchi to spectrum 5101
Added inchi to spectrum 5102
Added inchi to spectrum 5103
Added inchi to spectrum 5104
Added inchi to spectrum 5105
Added inchi to spectrum 5106
Added inchi to spectrum 5107
Added inchi to spectrum 5108
Added inchi to spectrum 5109
Added inchi to spectrum 5110
Added inchi to spectrum 5113
Added inchi to spectrum 5115
Added inchi to spectrum 5119
Added inchi to spectrum 5120
Added inchi to

Added inchi to spectrum 5649
Error for smiles: CC(=O)NC1CCCN2O[Al]345(ON(CCCC6NC(=O)C(CCCN(O3)C(\C=C(C)/CCOC1=O)=[O]4)NC6=O)C(C)=[O]5)[O]=C2\C=C(/C)CCO
Added inchi to spectrum 5650
Added inchi to spectrum 5651
Added inchi to spectrum 5652
Added inchi to spectrum 5653
Added inchi to spectrum 5769
Added inchi to spectrum 5777
Added inchi to spectrum 5778
Added inchi to spectrum 5791
Added inchi to spectrum 5792
Added inchi to spectrum 5796
Added inchi to spectrum 5797
Added inchi to spectrum 5798
Added inchi to spectrum 5799
Added inchi to spectrum 5800
Added inchi to spectrum 5801
Added inchi to spectrum 5810
Added inchi to spectrum 5813
Added inchi to spectrum 5819
Added inchi to spectrum 5820
Added inchi to spectrum 5823
Added inchi to spectrum 5824
Added inchi to spectrum 5825
Added inchi to spectrum 5826
Added inchi to spectrum 5827
Added inchi to spectrum 5828
Added inchi to spectrum 5829
Added inchi to spectrum 5830
Added inchi to spectrum 5831
Added inchi to spectrum 5832
Added i

...

Added inchi to spectrum 144045
Added inchi to spectrum 144046
Added inchi to spectrum 144047
Added inchi to spectrum 144048
Added inchi to spectrum 144049
Added inchi to spectrum 144050
Added inchi to spectrum 144051
Added inchi to spectrum 144052
Added inchi to spectrum 144053
Added inchi to spectrum 144054
Added inchi to spectrum 144055
Added inchi to spectrum 144056
Added inchi to spectrum 144057
Added inchi to spectrum 144058
Added inchi to spectrum 144059
Added inchi to spectrum 144060
Added inchi to spectrum 144061
Added inchi to spectrum 144062
Added inchi to spectrum 144063
Added inchi to spectrum 144064
Added inchi to spectrum 144065
Added inchi to spectrum 144066
Added inchi to spectrum 144067
Added inchi to spectrum 144068
Added inchi to spectrum 144069
Added inchi to spectrum 144070
Added inchi to spectrum 144071
Added inchi to spectrum 144072
Added inchi to spectrum 144073
Added inchi to spectrum 144074
Added inchi to spectrum 144075
Added inchi to spectrum 144076
Added in

Added inchi to spectrum 144407
Added inchi to spectrum 144408
Added inchi to spectrum 144409
Added inchi to spectrum 144410
Added inchi to spectrum 144411
Added inchi to spectrum 144412
Added inchi to spectrum 144413
Added inchi to spectrum 144414
Added inchi to spectrum 144415
Added inchi to spectrum 144416
Added inchi to spectrum 144417
Added inchi to spectrum 144418
Added inchi to spectrum 144419
Added inchi to spectrum 144420
Added inchi to spectrum 144421
Added inchi to spectrum 144422
Added inchi to spectrum 144423
Added inchi to spectrum 144424
Added inchi to spectrum 144425
Added inchi to spectrum 144426
Added inchi to spectrum 144427
Added inchi to spectrum 144428
Added inchi to spectrum 144429
Added inchi to spectrum 144430
Added inchi to spectrum 144431
Added inchi to spectrum 144432
Added inchi to spectrum 144433
Added inchi to spectrum 144434
Added inchi to spectrum 144435
Added inchi to spectrum 144436
Added inchi to spectrum 144437
Added inchi to spectrum 144438
Added in

Added inchi to spectrum 144800
Added inchi to spectrum 144801
Added inchi to spectrum 144802
Added inchi to spectrum 144803
Added inchi to spectrum 144804
Added inchi to spectrum 144805
Added inchi to spectrum 144806
Added inchi to spectrum 144807
Added inchi to spectrum 144808
Added inchi to spectrum 144809
Added inchi to spectrum 144810
Added inchi to spectrum 144811
Added inchi to spectrum 144812
Added inchi to spectrum 144813
Added inchi to spectrum 144814
Added inchi to spectrum 144815
Added inchi to spectrum 144816
Added inchi to spectrum 144817
Added inchi to spectrum 144818
Added inchi to spectrum 144819
Added inchi to spectrum 144820
Added inchi to spectrum 144821
Added inchi to spectrum 144822
Added inchi to spectrum 144823
Added inchi to spectrum 144824
Added inchi to spectrum 144825
Added inchi to spectrum 144826
Added inchi to spectrum 144827
Added inchi to spectrum 144828
Added inchi to spectrum 144829
Added inchi to spectrum 144830
Added inchi to spectrum 144831
Added in

Added inchi to spectrum 145257
Added inchi to spectrum 145258
Added inchi to spectrum 145259
Added inchi to spectrum 145260
Added inchi to spectrum 145261
Added inchi to spectrum 145262
Added inchi to spectrum 145263
Error for smiles: no data
Added inchi to spectrum 145264
Error for smiles: no data
Added inchi to spectrum 145265
Error for smiles: no data
Added inchi to spectrum 145266
Error for smiles: no data
Added inchi to spectrum 145267
Error for smiles: no data
Added inchi to spectrum 145268
Error for smiles: no data
Added inchi to spectrum 145269
Error for smiles: no data
Added inchi to spectrum 145270
Error for smiles: no data
Added inchi to spectrum 145271
Error for smiles: no data
Added inchi to spectrum 145272
Error for smiles: no data
Added inchi to spectrum 145273
Error for smiles: no data
Added inchi to spectrum 145274
Error for smiles: no data
Added inchi to spectrum 145275
Error for smiles: no data
Added inchi to spectrum 145276
Error for smiles: no data
Added inchi to s

Added inchi to spectrum 145503
Added inchi to spectrum 145504
Added inchi to spectrum 145505
Added inchi to spectrum 145506
Added inchi to spectrum 145507
Added inchi to spectrum 145508
Added inchi to spectrum 145509
Added inchi to spectrum 145510
Added inchi to spectrum 145511
Added inchi to spectrum 145512
Added inchi to spectrum 145513
Added inchi to spectrum 145514
Added inchi to spectrum 145515
Added inchi to spectrum 145516
Added inchi to spectrum 145517
Added inchi to spectrum 145518
Added inchi to spectrum 145519
Added inchi to spectrum 145520
Added inchi to spectrum 145521
Added inchi to spectrum 145522
Added inchi to spectrum 145523
Added inchi to spectrum 145524
Added inchi to spectrum 145525
Added inchi to spectrum 145526
Added inchi to spectrum 145527
Added inchi to spectrum 145528
Added inchi to spectrum 145529
Added inchi to spectrum 145530
Added inchi to spectrum 145531
Added inchi to spectrum 145532
Added inchi to spectrum 145533
Added inchi to spectrum 145534
Added in

In [67]:
collected_spectra_save[143150]

['BEGIN IONS\n',
 'PEPMASS=507.43\n',
 'CHARGE=1\n',
 'MSLEVEL=2\n',
 'SOURCE_INSTRUMENT=ESI-Flow-injection QqQ/MS\n',
 'FILENAME=respect_8_1_2014_GNPS_peaks.mgf\n',
 'SEQ=*..*\n',
 'IONMODE=Positive\n',
 'ORGANISM=RESPECT\n',
 "NAME=ReSpect:PS043401 Quercetin-3-O-glucose-6''-acetate|Quer-3-Glc-6pp-acetate|quercetin-3-O-beta-D-glucopyranosyl-6''-acetate|quercetin 3-(6''-acetylglucoside) [M+H]\n",
 'PI=Putative ReSpect Match\n',
 'DATACOLLECTOR=ReSpect\n',
 'SMILES=no data\n',
 'INCHI=N/A\n',
 'INCHIAUX=N/A\n',
 'PUBMED=N/A\n',
 'SUBMITUSER=mwang87\n',
 'TAGS=\n',
 'LIBRARYQUALITY=3\n',
 'SPECTRUMID=CCMSLIB00000215686\n',
 'SCANS=3522\n',
 '303.0\t87.0\n',
 '506.0\t275.0\n',
 '507.0\t999.0\n',
 '508.0\t299.0\n',
 'END IONS\n']

In [63]:
import sys

file_mgf = PATH_MS_DATA + 'all_gnps_inchi_added_191023.mgf'
with open(file_mgf, 'w') as f:
    for c in collected_spectra[:]:
        for l in c:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8

TITLE=(1S,8S,9S,10S,13R)-6,9,10-trimethyl-2-oxo-4,14-dioxatetracyclo[7.5.0.0�,��.0�,?]tetradeca-3(7),5-dien-8-yl acetate

TITLE=(1S,8S,9S,10S,13R)-6,9,10-trimethyl-2-oxo-4,14-dioxatetracyclo[7.5.0.0�,��.0�,?]tetradeca-3(7),5-dien-8-yl acetate

TITLE=(4E)-8-hydroxy-4-(1-hydroxypropan-2-ylidene)-10-oxatricyclo[7.2.1.0�,?]dodecane-8-carboxylic acid

TITLE=(4E)-8-hydroxy-4-(1-hydroxypropan-2-ylidene)-10-oxatricyclo[7.2.1.0�,?]dodecane-8-carboxylic acid

TITLE=(2R,3S,4S,9R,12R,13R,14S,15R,16R,17S)-3,4,12,13,15,16-hexahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-6-en-11-one

TITLE=(2R,3S,4S,9R,12R,13R,14S,15R,16R,17S)-3,4,12,13,15,16-hexahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-6-en-11-one

TITLE=15-(carbamoylmethyl)-10,11,23-trihydroxy-18-(3-methyl-2-oxopentanamido)-9,14,17-trioxo-N-[(1Z)-prop-1-en-1-yl]-8,13,16-triazatetracyclo[18.3.1.0�,?.0?,�?]tetracosa-1(23),2(7),3,5,20(24),21-hexaene-12-carboxamide

TITLE=15-(carbamoylmethyl

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,33-hexaoxaheptacyclo[28.7.1.0?,�?.0?,�?.0��,�?.0�?,��.0�?,�?]octatriaconta-1(38),11(16),12,14,17,19,21,34,36-nonaen-29-yl]acetic acid

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=2-[(5S,8R,11S,12S,13S,21R)-13,17,18-trihyd

...

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)o

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hy

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4R,5R,6S)-6-{[(2R,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydroxymethyl)-2-[(1'S,2S,4'S,5S,7'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}methyl)-5'-oxaspiro[oxolane-2,6'-pentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan]-18'-eneoxy]oxan-3-yl]oxy}-4,5-dihydroxy-2-(hydroxymethyl)oxan-3-yl]oxy}-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4R,5R,6S)-6-{[(2R,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydroxymethyl)-2-[(1'S,2S,4'S,5S,7'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}methyl)-5'-oxaspiro[oxolane-2,6'-pentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan]-18'-eneoxy]oxan-3-yl]oxy}-4,5-dihydroxy-2-(hydroxymethyl)oxan-3-yl]oxy}-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4R,5R,6S)-6-{[(2R,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydroxymethyl)-2-[(1'S,2S,4'S,5S,7'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydro

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl propanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl propanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl propanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl 2-methylpropanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo

## Replace charge values of "-1+" with "-1"

In [64]:
input_file = PATH_MS_DATA + 'all_gnps_inchi_added_191023.mgf'
output_file = open(ROOT + "\\Data\\GNPS_all\\" + "MS_data_allGNPS_191023_corrected.mgf", 'w')

with open(input_file, 'r') as file:
    for line in file:
        output_file.write(line.replace('-1+', '-1'))

In [65]:
output_file.close()